In [ ]:
import pandas as pd
import docx2txt
import os
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.corpus import stopwords

The next two cells are probably not needed after the first run. They just convert the word files to text files

In [ ]:
if not os.path.exists('text'):
    os.makedirs('text')

In [ ]:
for word_doc in os.listdir('./docx/'):
    filename,ext = os.path.splitext(word_doc)
    if ext == ".docx":
        raw_text = docx2txt.process(f'./docx/{word_doc}')
        with(open(f'./text/{filename}.txt','w')) as text_file:
            text_file.write(raw_text)

In [ ]:
corpus = []
filenames = []
for text_doc in os.listdir('./text'):
    filename,ext = os.path.splitext(text_doc)
    if ext == ".txt":
        filenames.append(f'{text_doc}')
        with(open(f'./text/{text_doc}','r',encoding="utf8", errors='ignore')) as f:
            corpus.append(f.read())

In [ ]:
corpus

In [ ]:
stop_words = set(stopwords.words('english'))

In [ ]:
text_vect = TfidfVectorizer(ngram_range=(1,2),stop_words=stop_words)
text_proc = text_vect.fit_transform(corpus)

In [ ]:
print(text_vect.get_feature_names())

In [ ]:
tfidf_array = text_proc.todense()
tfidf_values_list = tfidf_array.tolist()
tfidf_lists = pd.DataFrame(tfidf_values_list,
                           columns=text_vect.get_feature_names(),
                           index=filenames).T

In [ ]:
tfidf_lists.head()

In [ ]:
if not os.path.exists('csv'):
    os.makedirs('csv')

In [ ]:
for column_name in tfidf_lists.columns:
    new_file = column_name.replace('txt','csv')
    tfidf_lists[tfidf_lists[column_name]>0][column_name].sort_values(ascending=False).to_csv(f'./csv/{new_file}')

[LDA Stuff](https://towardsdatascience.com/2-latent-methods-for-dimension-reduction-and-topic-modeling-20ff6d7d547)
[LDA Stuff 2](https://medium.com/analytics-vidhya/topic-modelling-using-latent-dirichlet-allocation-in-scikit-learn-7daf770406c4)

In [ ]:
text_cvect = CountVectorizer(max_df = 0.95, min_df = 2, stop_words = stop_words)
text_cproc = text_cvect.fit_transform(corpus)

In [ ]:
text_lda = LatentDirichletAllocation(n_components = 10, random_state = 42)
text_lda.fit(text_cproc)

In [ ]:
for index, topic in enumerate(text_lda.components_):
    print(f'Top 15 words for Topic #{index}')
    print([text_cvect.get_feature_names()[i] for i in topic.argsort()[-15:]])
    print('\n')

In [ ]:
text_lda.components_